In [3]:
from mods import DatabaseConnector 
import mysql.connector
import os
import pandas as pd

In [4]:
# Create a new instance of DatabaseConnector
myDB = DatabaseConnector('localhost','root','root','final_project')
directory = r"C:\Users\marno\Wiley Edge\Final_Project\Code\data"

# Data Cleaning and Transformation

In [5]:
# Create a dictionary of table names and file paths
def get_csv_files(directory: str) -> dict[str, str]:
    tables = {}
    # For each file in a directory, add the file name (excluding the extension) and the file path to the dictionary
    for file in os.listdir(directory):
        if file.endswith(".csv"):
            table_name = file.split(".")[0]
            file_path = os.path.join(directory, file).replace("\\", "/")
            tables[table_name] = file_path
    return tables


# Function to populate each table in the database by using the csv files in the directory.
def load_data_to_table(csv_file_path, table_name, myDB):
    try:
        # Connect to the database
        myDB.connect()
                
        # Build the SQL query
        query = f'LOAD DATA LOCAL INFILE "{csv_file_path}" INTO TABLE {table_name} FIELDS TERMINATED BY "," ENCLOSED BY \'"\' IGNORE 1 ROWS'

        # check if table is already populated
        update_query = f"SELECT COUNT(*) FROM {table_name}"
        result = myDB.execute_query(update_query)
        
        # If the table is not populated, load the data
        if result[0][0] == 0:
            myDB.update_query(query)
            print(f"Data from {csv_file_path} loaded into {table_name} successfully.")
        else:
            print(f"Table {table_name} is already populated.")

        print(f"Data from {csv_file_path} loaded into {table_name} successfully.")
    except Exception as e:
        print(f"Error loading data: {e}")
    finally:
        # Close the connection
        myDB.close_connection()
        
# Implement the function for each table to load all the data
def load_all_data():
    for table_name, file_path in get_csv_files(directory).items():
        load_data_to_table(file_path, table_name, myDB)
    
    
def import_data():
    for table_name in get_csv_files(directory).keys():
        myDB.import_data(table_name)

In [6]:
import_data()

Connected to database
Data from customers imported successfully.
Connected to database
Data from geolocation imported successfully.
Connected to database
Data from orders imported successfully.
Connected to database
Data from order_items imported successfully.
Connected to database
Data from order_payments imported successfully.
Connected to database
Data from order_reviews imported successfully.
Connected to database
Data from products imported successfully.
Connected to database
Data from product_category_translation imported successfully.
Connected to database
Data from sellers imported successfully.


In [7]:
def get_dataframes():
    return myDB.dataframes

def update_csv_files(dataframes_dict, directory_path):

    # Iterate through the dictionary
    for title, dataframe in dataframes_dict.items():
        # Construct the CSV file path
        csv_file_path = os.path.join(directory_path, f'{title}.csv')

        # Check if the CSV file already exists
        if os.path.isfile(csv_file_path):
            # If it exists, update the CSV file with the new dataframe
            dataframe.to_csv(csv_file_path, index=False)
            print(f'{title}.csv updated successfully.')
        else:
            # If it doesn't exist, create a new CSV file
            dataframe.to_csv(csv_file_path, index=False)
            print(f'{title}.csv created successfully.')
            
update_csv_files(get_dataframes(), directory)


customers.csv updated successfully.
geolocation.csv updated successfully.
orders.csv updated successfully.
order_items.csv updated successfully.
order_payments.csv updated successfully.
order_reviews.csv updated successfully.
products.csv updated successfully.
product_category_translation.csv updated successfully.
sellers.csv updated successfully.


In [8]:
import pandas as pd

# Replace the Portuguese category names with English names
def replace_category_names(products_df, translations_df):
    # Merge the Products table with the translations table based on the product category name
    merged_df = pd.merge(products_df, translations_df, on='product_category_name', how='left')

    # Replace the original category name column with the English names
    merged_df['product_category_name'] = merged_df['english_category_name']

    # Drop the redundant translated column
    merged_df = merged_df.drop('english_category_name', axis=1)

    return merged_df

# Call the function to replace category names
updated_products_df = replace_category_names(myDB.dataframes['products'], myDB.dataframes['product_category_translation'])

# remove carriage returns from the product_category_name column
updated_products_df['product_category_name'] = updated_products_df['product_category_name'].str.replace('\r', '')

# replace the product dataframe in the dictionary with the updated dataframe
myDB.dataframes['products'] = updated_products_df




In [9]:
# Update the products csv file with the new dataframe

update_csv_files(get_dataframes(), directory)



customers.csv updated successfully.
geolocation.csv updated successfully.
orders.csv updated successfully.
order_items.csv updated successfully.
order_payments.csv updated successfully.
order_reviews.csv updated successfully.
products.csv updated successfully.
product_category_translation.csv updated successfully.
sellers.csv updated successfully.


In [10]:
# add a new column in geolocations table to store the full state name instead of the abbreviation
# get dictionary of state abbreviations and full names
def get_state_dict():
    return {
        'AC': 'Acre',
        'AL': 'Alagoas',
        'AP': 'Amapá',
        'AM': 'Amazonas',
        'BA': 'Bahia',
        'CE': 'Ceará',
        'DF': 'Distrito Federal',
        'ES': 'Espírito Santo',
        'GO': 'Goiás',
        'MA': 'Maranhão',
        'MT': 'Mato Grosso',
        'MS': 'Mato Grosso do Sul',
        'MG': 'Minas Gerais',
        'PA': 'Pará',
        'PB': 'Paraíba',
        'PR': 'Paraná',
        'PE': 'Pernambuco',
        'PI': 'Piauí',
        'RJ': 'Rio de Janeiro',
        'RN': 'Rio Grande do Norte',
        'RS': 'Rio Grande do Sul',
        'RO': 'Rondônia',
        'RR': 'Roraima',
        'SC': 'Santa Catarina',
        'SP': 'São Paulo',
        'SE': 'Sergipe',
        'TO': 'Tocantins'
    }
    
# Create a new column in the geolocation dataframe to store the full state name
def add_state_name(geolocation_df):
    # Get the state dictionary
    state_dict = get_state_dict()

    # Create a new column in the dataframe to store the full state name
    geolocation_df['state_name'] = geolocation_df['state'].map(state_dict)

    return geolocation_df

# Call the function to add the state name column
updated_geolocation_df = add_state_name(myDB.dataframes['geolocation'])

# update the csv file with the new dataframe
myDB.dataframes['geolocation'] = updated_geolocation_df

update_csv_files(get_dataframes(), directory)



customers.csv updated successfully.
geolocation.csv updated successfully.
orders.csv updated successfully.
order_items.csv updated successfully.
order_payments.csv updated successfully.
order_reviews.csv updated successfully.
products.csv updated successfully.
product_category_translation.csv updated successfully.
sellers.csv updated successfully.


In [11]:
# add a new column to the orders table to see the time taken between order purchase timestamp and order delivered
def add_delivery_time(orders_df):
    # Convert the purchase and delivery timestamps to datetime
    orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
    orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])

    # Calculate the time taken to deliver the order
    orders_df['delivery_time'] = orders_df['order_delivered_customer_date'] - orders_df['order_purchase_timestamp']

    # Convert the time taken to days
    orders_df['delivery_time'] = orders_df['delivery_time'].dt.days

    return orders_df

# Call the function to add the delivery time column
updated_orders_df = add_delivery_time(myDB.dataframes['orders'])


# update the csv file with the new dataframe
myDB.dataframes['orders'] = updated_orders_df


# count null values in in delivery_time column
myDB.dataframes['orders']['delivery_time'].isnull().sum()
# remove rows with null values in delivery_time column
myDB.dataframes['orders'] = myDB.dataframes['orders'][myDB.dataframes['orders']['delivery_time'].notna()]
# make delivery_time column an integer
myDB.dataframes['orders']['delivery_time'] = myDB.dataframes['orders']['delivery_time'].astype(int)
# update the csv file with the new dataframe
update_csv_files(get_dataframes(), directory)


C:\Users\marno\AppData\Local\Temp\ipykernel_49712\1054106570.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myDB.dataframes['orders']['delivery_time'] = myDB.dataframes['orders']['delivery_time'].astype(int)


customers.csv updated successfully.
geolocation.csv updated successfully.
orders.csv updated successfully.
order_items.csv updated successfully.
order_payments.csv updated successfully.
order_reviews.csv updated successfully.
products.csv updated successfully.
product_category_translation.csv updated successfully.
sellers.csv updated successfully.


## Data exploration and descriptive analysis

In [12]:
# print pandas info for each table
for table_name, dataframe in myDB.dataframes.items():
    print(f'Table Name: {table_name}')
    print(dataframe.info())
    print('\n')


Table Name: customers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None


Table Name: geolocation
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19015 entries, 0 to 19014
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   zip_code_prefix  19015 non-null  int64 
 1   lat              19015 non-null  object
 2   lng              19015 non-null  object
 3   city             19015 non-null  object
 4   state            19015 non-n

In [13]:
# print first 5 rows of each table
myDB.dataframes['customers'].head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,29830,nova venecia,ES
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,4841,sao paulo,SP


In [14]:
myDB.dataframes['geolocation'].head()

,zip_code_prefix,lat,lng,city,state,state_name
0,1001,-23.549292,-46.633559,sao paulo,SP,São Paulo
1,1002,-23.548318,-46.635421,sao paulo,SP,São Paulo
2,1003,-23.549032,-46.635313,sao paulo,SP,São Paulo
3,1004,-23.550116,-46.635122,sao paulo,SP,São Paulo
4,1005,-23.549819,-46.635606,sao paulo,SP,São Paulo


In [15]:
myDB.dataframes['order_items'].head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [16]:
myDB.dataframes['order_payments'].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,credit_card,2,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,credit_card,3,259.83
2,000229ec398224ef6ca0657da4fc703e,1,credit_card,5,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,credit_card,2,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,credit_card,3,218.04


In [17]:
myDB.dataframes['order_reviews'].head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0001239bc1de2e33cb583967c2ca4c67,fc046d7776171871436844218f817d7d,5,,,2018-03-20,2018-03-20 18:36:04
1,0001cc6860aeaf5b9017fe4131a52e62,d4665434b01caa9dc3e3e78b3eb3593e,5,,,2018-06-22,2018-06-26 13:51:29
2,00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5,Entrega rápida!,A entrega foi super rápida e o pendente é lind...,2018-04-25,2018-04-26 14:55:36
3,00032b0141443497c898b3093690af51,04fb47576993a3cb0c12d4b25eab6e4e,5,,,2017-05-30,2017-06-01 23:28:55
4,00034d88989f9a4c393bdcaec301537f,5f358d797a49fe2f24352f73426215f6,5,,,2017-08-12,2017-08-13 19:56:53


In [18]:
myDB.dataframes['orders'].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,7
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,16
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,7
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,6
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,25


In [19]:
myDB.dataframes['products'].head()

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00066f42aeeb9f3007548bb9d3f33c38,perfumery,300,20,16,16
1,00088930e925c41fd95ebfe695fd2655,auto,1225,55,10,26
2,0009406fd7479715e4bef61dd91f2462,bed_bath_table,300,45,15,35
3,000b8f95fcb9e0096488278317764d19,housewares,550,19,24,12
4,000d9be29b5207b54e86aa1b1ac54872,watches_gifts,250,22,11,15


In [20]:
myDB.dataframes['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0015a82c2db000af6aaaf3ae2ecb0532,9080,santo andre,SP
1,001cca7ae9ae17fb1caed9dfb1094831,29156,cariacica,ES
2,001e6ad469a905060d959994f1b41e4f,24754,sao goncalo,RJ
3,002100f778ceb8431b7a1020ff7ab48f,14405,franca,SP
4,003554e2dce176b5555353e4f3555ac8,74565,goiania,GO


In [21]:
# Check for null values in the order_reviews table
myDB.dataframes['order_reviews'].isnull().sum()

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       1
review_answer_timestamp    1
dtype: int64

In [22]:
# check for null values in each table
for table_name, dataframe in myDB.dataframes.items():
    print(f'Table Name: {table_name}')
    print(dataframe.isnull().sum())
    print('\n')



Table Name: customers
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


Table Name: geolocation
zip_code_prefix    0
lat                0
lng                0
city               0
state              0
state_name         0
dtype: int64


Table Name: orders
order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      1
order_delivered_customer_date     0
order_estimated_delivery_date     0
delivery_time                     0
dtype: int64


Table Name: order_items
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


Table Name: order_payments
order_id                0
payment_sequential     

In [23]:
# conduct relavant data analysis with pandas
customers_df = myDB.dataframes['customers']
orders_df = myDB.dataframes['orders']
order_items_df = myDB.dataframes['order_items']
products_df = myDB.dataframes['products']
sellers_df = myDB.dataframes['sellers']
order_payments_df = myDB.dataframes['order_payments']
order_reviews_df = myDB.dataframes['order_reviews']
product_category_translation_df = myDB.dataframes['product_category_translation']
